In [ ]:
from pathlib import Path

try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB and Path.cwd().name == "content":
    if not Path("quantum-jam-chirimbolo").exists():
        !git clone https://github.com/segusantos/quantum-jam-chirimbolo.git
        !pip install quantum-jam-chirimbolo/
    %cd quantum-jam-chirimbolo


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import HTML, display
import ipywidgets as widgets

from src import (
    BB84Protocol,
    BB84Parameters,
    NoiseChannel,
    CascadeErrorCorrector,
    PrivacyAmplifier,
    NOISE_PRESETS,
    run_bb84,
    style_result_table,
    key_summary,
    detection_summary,
    split_after_detection,
    sweep_noise,
    render_noise_curves,
    format_key_preview,
    configure_noise_slider,
    bind_controls,
)

plt.style.use("seaborn-v0_8-darkgrid")
pd.options.display.max_rows = 200
pd.options.display.max_columns = 20

# Protocolo BB84 con Qiskit

Exploramos un flujo completo de distribución de claves cuánticas BB84, desde la preparación de qubits hasta la corrección de errores y la amplificación de privacidad. Cada sección del cuaderno incorpora controles interactivos para experimentar con los parámetros del protocolo y visualizar métricas clave.


## Arquitectura del prototipo

- `BB84Protocol` orquesta la generación de estados, el ataque de Eve y las medidas de Bob empleando `AerSimulator`.
- `NoiseChannel` define canales de ruido configurables (depolarizante, bit/phase flip, amortiguamiento, etc.).
- `CascadeErrorCorrector` aplica corrección de errores clásica sobre la clave cribada.
- `PrivacyAmplifier` reduce la clave para absorber la fuga de información estimada.

La celdas siguientes usan estas primitivas para construir una narrativa interactiva en cuatro etapas.


## Parte 1 — Ejecución básica de BB84

Ajusta el número de bits generados por Alice y observa cómo se producen las coincidencias de bases y la clave cribada compartida.


In [ ]:
part1_slider = widgets.IntSlider(value=12, min=4, max=64, step=1, description="Bits (N)")
part1_output = widgets.Output()

def render_part1(num_bits):
    result = run_bb84(
        num_bits=num_bits,
        seed_value="",
        eve=False,
        eve_prob=0.0,
        noise_option="Sin ruido",
        noise_value=0.0,
        readout0=0.0,
        readout1=0.0,
    )
    display(key_summary(result))
    display(style_result_table(result))

bind_controls({"num_bits": part1_slider}, render_part1, part1_output)
display(widgets.VBox([part1_slider, part1_output]))


## Parte 2 — Ataque de Eve y muestreo de detección

Se incorpora una semilla configurable para reproducir experimentos, además de un ataque de interceptación-resend por parte de Eve. Ajusta la probabilidad de interceptación y la cantidad de bits `m` usados para detección.


In [ ]:
part2_num_bits = widgets.IntSlider(value=24, min=8, max=128, step=2, description="Bits (N)")
part2_seed = widgets.Text(value="", description="Semilla", placeholder="vacio = aleatorio")
part2_eve = widgets.Checkbox(value=True, description="Eve presente")
part2_eve_prob = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.05, description="P(intercept)")
part2_detection = widgets.IntSlider(value=12, min=2, max=64, step=1, description="m (deteccion)")
part2_output = widgets.Output()


def _toggle_eve_slider(change):
    part2_eve_prob.disabled = not change["new"]


part2_eve.observe(_toggle_eve_slider, names="value")
part2_eve_prob.disabled = not part2_eve.value


def render_part2(num_bits, seed_value, eve, eve_prob, detection_bits):
    result = run_bb84(
        num_bits=num_bits,
        seed_value=seed_value,
        eve=eve,
        eve_prob=eve_prob,
        noise_option="Sin ruido",
        noise_value=0.0,
        readout0=0.0,
        readout1=0.0,
    )
    display(key_summary(result))
    display(style_result_table(result))
    display(detection_summary(result, detection_bits))


controls_part2 = {
    "num_bits": part2_num_bits,
    "seed_value": part2_seed,
    "eve": part2_eve,
    "eve_prob": part2_eve_prob,
    "detection_bits": part2_detection,
}

bind_controls(controls_part2, render_part2, part2_output)
display(
    widgets.VBox(
        [
            widgets.HBox([part2_num_bits, part2_detection]),
            widgets.HBox([part2_seed, part2_eve, part2_eve_prob]),
            part2_output,
        ]
    )
)


## Parte 3 — Explorando el impacto del ruido en BB84

Esta sección extiende la anterior e introduce canales de ruido configurables mediante `AerSimulator`. Ajusta el tipo de ruido y observa cómo varían el QBER y la probabilidad de detección.


In [ ]:
part3_num_bits = widgets.IntSlider(value=48, min=16, max=256, step=4, description="Bits (N)")
part3_seed = widgets.Text(value="", description="Semilla", placeholder="vacio = aleatorio")
part3_eve = widgets.Checkbox(value=True, description="Eve presente")
part3_eve_prob = widgets.FloatSlider(value=0.3, min=0.0, max=1.0, step=0.05, description="P(intercept)")
part3_detection = widgets.IntSlider(value=16, min=4, max=96, step=2, description="m (deteccion)")
part3_noise = widgets.Dropdown(options=list(NOISE_PRESETS.keys()), value="Depolarizante", description="Ruido")
part3_noise_value = widgets.FloatSlider(value=0.1, min=0.0, max=0.3, step=0.01, description="p")
part3_readout0 = widgets.FloatSlider(value=0.0, min=0.0, max=0.2, step=0.01, description="p0->1")
part3_readout1 = widgets.FloatSlider(value=0.0, min=0.0, max=0.2, step=0.01, description="p1->0")
part3_points = widgets.IntSlider(value=7, min=3, max=15, step=1, description="barrido")
part3_output = widgets.Output()


configure_noise_slider(part3_noise, part3_noise_value)


def _toggle_part3_eve(change):
    part3_eve_prob.disabled = not change["new"]


part3_eve.observe(_toggle_part3_eve, names="value")
part3_eve_prob.disabled = not part3_eve.value


def render_part3(
    num_bits,
    seed_value,
    eve,
    eve_prob,
    detection_bits,
    noise_option,
    noise_value,
    readout0,
    readout1,
    sweep_points,
):
    result = run_bb84(
        num_bits=num_bits,
        seed_value=seed_value,
        eve=eve,
        eve_prob=eve_prob,
        noise_option=noise_option,
        noise_value=noise_value,
        readout0=readout0,
        readout1=readout1,
    )
    display(key_summary(result))
    sample = min(detection_bits, result.sifted_key_length())
    info = (
        f"<p>Bits cribados: <strong>{result.sifted_key_length()}</strong> | "
        f"QBER: <strong>{result.qber:.4f}</strong> | Muestreo m: {sample}</p>"
    )
    display(HTML(info))
    display(style_result_table(result))
    display(detection_summary(result, detection_bits))

    preset = NOISE_PRESETS[noise_option]
    data = []
    if preset["param"] and sweep_points > 1:
        upper = preset["max"] if preset["max"] > 0 else max(noise_value, 0.1)
        upper = max(upper, noise_value if noise_value > 0 else 0.1)
        values = np.linspace(0.0, upper, sweep_points)
        values = np.unique(np.append(values, noise_value))
        data = sweep_noise(
            num_bits,
            seed_value,
            eve,
            eve_prob,
            noise_option,
            values,
            readout0,
            readout1,
            detection_bits,
        )
    fig = render_noise_curves(data, preset["label"] or "valor")
    if fig is not None:
        display(fig)
        plt.close(fig)


controls_part3 = {
    "num_bits": part3_num_bits,
    "seed_value": part3_seed,
    "eve": part3_eve,
    "eve_prob": part3_eve_prob,
    "detection_bits": part3_detection,
    "noise_option": part3_noise,
    "noise_value": part3_noise_value,
    "readout0": part3_readout0,
    "readout1": part3_readout1,
    "sweep_points": part3_points,
}

bind_controls(controls_part3, render_part3, part3_output)
display(
    widgets.VBox(
        [
            widgets.HBox([part3_num_bits, part3_detection, part3_points]),
            widgets.HBox([part3_seed, part3_eve, part3_eve_prob]),
            widgets.HBox([part3_noise, part3_noise_value]),
            widgets.HBox([part3_readout0, part3_readout1]),
            part3_output,
        ]
    )
)


## Parte 4 — Corrección de errores y amplificación de privacidad

Aplicamos el algoritmo Cascade para corregir discrepancias residuales y posteriormente ejecutamos privacidad amplificada con SHAKE-256 truncado.


In [ ]:
part4_num_bits = widgets.IntSlider(value=64, min=16, max=256, step=4, description="Bits (N)")
part4_seed = widgets.Text(value="", description="Semilla", placeholder="vacio = aleatorio")
part4_eve = widgets.Checkbox(value=True, description="Eve presente")
part4_eve_prob = widgets.FloatSlider(value=0.4, min=0.0, max=1.0, step=0.05, description="P(intercept)")
part4_detection = widgets.IntSlider(value=20, min=4, max=128, step=2, description="m (deteccion)")
part4_noise = widgets.Dropdown(options=list(NOISE_PRESETS.keys()), value="Depolarizante", description="Ruido")
part4_noise_value = widgets.FloatSlider(value=0.05, min=0.0, max=0.3, step=0.01, description="p")
part4_readout0 = widgets.FloatSlider(value=0.0, min=0.0, max=0.2, step=0.01, description="p0->1")
part4_readout1 = widgets.FloatSlider(value=0.0, min=0.0, max=0.2, step=0.01, description="p1->0")
part4_block = widgets.IntSlider(value=4, min=2, max=16, step=1, description="bloque")
part4_rounds = widgets.IntSlider(value=3, min=1, max=5, step=1, description="rondas")
part4_security = widgets.IntSlider(value=40, min=10, max=80, step=5, description="seguridad")
part4_target = widgets.IntSlider(value=0, min=0, max=128, step=4, description="target (0 auto)")
part4_output = widgets.Output()


configure_noise_slider(part4_noise, part4_noise_value)


def _toggle_part4_eve(change):
    part4_eve_prob.disabled = not change["new"]


part4_eve.observe(_toggle_part4_eve, names="value")
part4_eve_prob.disabled = not part4_eve.value


def render_part4(
    num_bits,
    seed_value,
    eve,
    eve_prob,
    detection_bits,
    noise_option,
    noise_value,
    readout0,
    readout1,
    bloque,
    rondas,
    seguridad,
    target_bits,
):
    result = run_bb84(
        num_bits=num_bits,
        seed_value=seed_value,
        eve=eve,
        eve_prob=eve_prob,
        noise_option=noise_option,
        noise_value=noise_value,
        readout0=readout0,
        readout1=readout1,
    )
    display(key_summary(result))
    display(style_result_table(result))
    display(detection_summary(result, detection_bits))

    alice_rest, bob_rest, alice_sample, bob_sample = split_after_detection(result, detection_bits)
    sample = len(alice_sample)
    sample_mismatches = sum(1 for a, b in zip(alice_sample, bob_sample) if a != b)
    detection_lines = [
        "MUESTREO PARA DETECCION",
        "------------------------------------------",
        f"m                 : {sample}",
        f"Discrepancias     : {sample_mismatches}",
        f"Alice (m)         : {format_key_preview(alice_sample, 48)}",
        f"Bob   (m)         : {format_key_preview(bob_sample, 48)}",
    ]
    display(HTML("<pre>" + "\n".join(detection_lines) + "</pre>"))

    if not alice_rest:
        display(HTML("<p><strong>No quedan bits tras el muestreo. Disminuye m o incrementa N.</strong></p>"))
        return

    corrector = CascadeErrorCorrector(block_size=bloque, rounds=rondas)
    cascade = corrector.correct(alice_rest, bob_rest)
    cascade_lines = [
        "CASCADE (correccion)",
        "------------------------------------------",
        f"Longitud entrada  : {len(alice_rest)}",
        f"Bloque inicial    : {bloque}",
        f"Rondas            : {rondas}",
        f"Correcciones      : {len(cascade.corrections)}",
        f"Bits fugados      : {cascade.leakage_bits}",
        f"Errores residuales: {cascade.residual_errors}",
        f"Clave corregida   : {format_key_preview(cascade.corrected_key, 64)}",
    ]
    display(HTML("<pre>" + "\n".join(cascade_lines) + "</pre>"))

    amplifier = PrivacyAmplifier(security_parameter=seguridad)
    target = None if target_bits <= 0 else target_bits
    privacy = amplifier.apply(cascade.corrected_key, cascade.leakage_bits, target_length=target)
    privacy_lines = [
        "PRIVACY AMPLIFICATION",
        "------------------------------------------",
        f"Hash              : {privacy.hash_function}",
        f"Seguridad (bits)  : {seguridad}",
        f"Target solicitado : {target if target is not None else 'auto'}",
        f"Longitud final    : {privacy.target_length}",
        f"Bits descartados  : {privacy.discarded_bits}",
        f"Clave final       : {format_key_preview(privacy.final_key, 64)}",
    ]
    display(HTML("<pre>" + "\n".join(privacy_lines) + "</pre>"))

    if cascade.residual_errors > 0:
        display(HTML("<p style='color:#c62828;font-weight:600;'>Persisten errores tras Cascade; aumenta rondas o m.</p>"))


controls_part4 = {
    "num_bits": part4_num_bits,
    "seed_value": part4_seed,
    "eve": part4_eve,
    "eve_prob": part4_eve_prob,
    "detection_bits": part4_detection,
    "noise_option": part4_noise,
    "noise_value": part4_noise_value,
    "readout0": part4_readout0,
    "readout1": part4_readout1,
    "bloque": part4_block,
    "rondas": part4_rounds,
    "seguridad": part4_security,
    "target_bits": part4_target,
}

bind_controls(controls_part4, render_part4, part4_output)
display(
    widgets.VBox(
        [
            widgets.HBox([part4_num_bits, part4_detection]),
            widgets.HBox([part4_seed, part4_eve, part4_eve_prob]),
            widgets.HBox([part4_noise, part4_noise_value]),
            widgets.HBox([part4_readout0, part4_readout1]),
            widgets.HBox([part4_block, part4_rounds, part4_security, part4_target]),
            part4_output,
        ]
    )
)


## Conclusiones y próximos pasos

- Explora valores más altos de `N` para obtener estadísticas más estables.
- Ajusta `m`, las rondas de Cascade y el parámetro de seguridad para equilibrar detección y longitud final.
- Conecta el cuaderno a un backend real de IBM Quantum para contrastar con hardware físico.
